In [444]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd

### Constant Declaration

In [445]:
stock_number = 5
minimum_score = 75

upper_th = {
    "rsi" : 70,
    "10di-" : 18,
    "25di-" : 18,
}

lower_th = {
    "10di+" : 25,
    "25di+":25,
    "25adx":20,
    "10adx":25,
}

In [446]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'}

tv_url = "https://id.tradingview.com/markets/stocks-indonesia/market-movers-active/"
tv_index_class = "apply-common-tooltip tickerNameBox-GrtoTeat tickerName-GrtoTeat"

yh_url = "https://finance.yahoo.com/quote/"

### Stock Index Fetching

In [447]:
tv_page = req.get(tv_url,headers=headers)
soup = BeautifulSoup(tv_page.content, 'html.parser')

In [448]:
main_table = soup.find('table')
stock_rows = main_table.find_all('a',class_="apply-common-tooltip tickerNameBox-GrtoTeat tickerName-GrtoTeat")
stocks = [row.get_text() for row in stock_rows]

print("Stocks: ",stocks)

Stocks:  ['BBCA', 'BBRI', 'BMRI', 'ANTM', 'TLKM', 'BBNI', 'BREN', 'ASII', 'AMMN', 'ADRO', 'TPIA', 'PTRO', 'PANI', 'UNTR', 'WIKA', 'GOTO', 'BRPT', 'INCO', 'FILM', 'AMRT', 'INDF', 'MDKA', 'TOBA', 'CMRY', 'PNLF', 'CPIN', 'KLBF', 'SSIA', 'ABMM', 'BDKR', 'INKP', 'CUAN', 'BSBK', 'ICBP', 'MIKA', 'PGAS', 'UNVR', 'SRTG', 'ESSA', 'ISAT', 'BRIS', 'SMRA', 'GEMS', 'BOGA', 'DSSA', 'TCPI', 'MAPA', 'NISP', 'PSAB', 'CTRA', 'PTBA', 'SIDO', 'EXCL', 'MEDC', 'ARTO', 'GJTL', 'SMIL', 'BCAP', 'BSDE', 'EURO', 'SMGR', 'PNBN', 'ITMG', 'BBTN', 'SILO', 'ADMR', 'PWON', 'DSNG', 'MAPI', 'JSMR', 'JPFA', 'KIJA', 'TOWR', 'CMNT', 'BUMI', 'INET', 'OASA', 'PTPP', 'MYOR', 'BUKA', 'HRUM', 'BIPI', 'MBMA', 'GGRM', 'AKRA', 'ACES', 'BSSR', 'ERAA', 'WIFI', 'HATM', 'TSPC', 'MTEL', 'ADHI', 'APEX', 'TKIM', 'HEAL', 'BFIN', 'SSMS', 'AUTO', 'NEST']


### Stock Price Mining

##### Fetching Functions Declaration

In [449]:
def convert_text_to_float(text):
    return float(text.replace(",",""))

def parse_price_per_row(columns):
    return [convert_text_to_float(col.get_text()) for col in columns]

In [450]:
def fetch_stock_rows(stock):
    yh_page = req.get(yh_url+stock+".JK/history",headers=headers)
    soup = BeautifulSoup(yh_page.content, 'html.parser')
    
    main_table = soup.find('table')
    price_rows = main_table.find_all('tr')[1:32]
    return price_rows

def get_stock_df(price_rows):
    data_rows = []
    
    for row in price_rows:
        columns = row.find_all('td')
        columns = columns[1:5]
    
        try : 
            open, high, low, close = parse_price_per_row(columns)
            data_rows.append({'Open':open,'High':high,'Low':low,'Close':close})
        except :
            pass

    df = pd.DataFrame(data_rows)
    reversed_df = df.iloc[::-1].reset_index(drop=True)
        
    return reversed_df

##### Assessment Functions Declaration

In [451]:
def get_true_range(df):
    tr_1 = df['High'] - df['Low']
    tr_2 = abs(df['High'] - df['Close'].shift())
    tr_3 = abs(df['Low'] - df['Close'].shift())
    
    tr_final = pd.concat([tr_1,tr_2,tr_3],axis=1).max(axis=1)
    return tr_final

In [452]:
def get_rsi(df, period=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def get_atr(df, period=14):
    tr = get_true_range(df)
    return tr.rolling(window=period).mean()

def get_dis(df, period=14):
    atr = get_atr(df, period)
    
    up = df['High'] - df['High'].shift()
    down = df['Low'].shift() - df['Low']
    
    plus_dm = (up.where(up > down, 0)).rolling(window=period).mean()
    minus_dm = (-down.where(down > up, 0)).rolling(window=period).mean()
    
    plus_di = 100 * (plus_dm / atr)
    minus_di = 100 * (minus_dm / atr)
    
    return plus_di, minus_di

def get_adx(df, period=14):
    plus_di, minus_di = get_dis(df, period)
    dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
    return dx.rolling(window=period).mean()

In [453]:
def assess_rsi(df,period=14):
    rsi = get_rsi(df, period)
    
    return rsi.iloc[-1] <= upper_th['rsi']

def assess_dis(df,period=14):
    plus_di, minus_di = get_dis(df, period)
    return plus_di.iloc[-1] >= lower_th['10di+'] and minus_di.iloc[-1] <= upper_th['10di-']

def assess_adx(df,period=14):
    adx = get_adx(df, period)
    return adx.iloc[-1] >= lower_th['25adx']

def assess_stock(df):    
    rsi = assess_rsi(df)
    short_dis = assess_dis(df,period=10)
    long_dis = assess_dis(df,period=25)
    short_adx = assess_adx(df,period=10)
    long_adx = assess_adx(df,period=25)
    
    return rsi, short_dis, long_dis, short_adx, long_adx

#####  Stock Filtration

In [454]:
found_stocks = []
assessed_stocks = []

for stock in stocks : 
    if len(found_stocks) >= stock_number:
        break
    
    stock_price_rows = fetch_stock_rows(stock)
    stock_df = get_stock_df(stock_price_rows)
        
    try:
        assessment = list(assess_stock(stock_df))
        
        score = (sum(assessment)/len(assessment))*100
        if score >= minimum_score:
            found_stocks.append(stock)
            
        assessed_stocks.append({'Stock':stock,'Score':score})
    except:
        pass

      Open     High     Low    Close
0  10050.0  10125.0  9975.0  10125.0
     Open    High     Low   Close
0  4870.0  4910.0  4850.0  4900.0
     Open    High     Low   Close
0  6500.0  6525.0  6450.0  6525.0


KeyboardInterrupt: 

##### Found Stock Display

In [ ]:
print(found_stocks)

['BRPT', 'FILM', 'SSIA']
